In [7]:
import pandas as pd
import os
import re
import spacy
import logging


In [1]:
#!python -m spacy download en_core_web_sm
#!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
     - -------------------------------------- 0.4/12.8 MB 3.8 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 7.4 MB/s eta 0:00:02
     ---- ----------------------------------- 1.6/12.8 MB 9.1 MB/s eta 0:00:02
     ------- -------------------------------- 2.5/12.8 MB 12.5 MB/s eta 0:00:01
     ----------- ---------------------------- 3.6/12.8 MB 14.4 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 20.6 MB/s eta 0:00:01
     ----------------------- ---------------- 7.7/12.8 MB 23.4 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 38.5 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 50.4 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 43.7 MB/s eta 0:00:00
[+] Download and installation successful
You can no

DEPRECATION: Loading egg at c:\users\nmadali\appdata\local\anaconda3\lib\site-packages\pyasn1_modules-0.3.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     --------------------------------------- 0.0/16.3 MB 640.0 kB/s eta 0:00:26
     ---------------------------------------- 0.1/16.3 MB 1.4 MB/s eta 0:00:12
      --------------------------------------- 0.3/16.3 MB 3.1 MB/s eta 0:00:06
     - -------------------------------------- 0.8/16.3 MB 4.9 MB/s eta 0:00:04
     --- ------------------------------------ 1.4/16.3 MB 6.9 MB/s eta 0:00:03
     ----- ---------------------------------- 2.1/16.3 MB 8.2 MB/s eta 0:00:02
     ------- -------------------------------- 3.0/16.3 MB 10.8 MB/s eta 0:00:02
     --------- ------------------------------ 4.0/16.3 MB 12.3 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 15.5 MB/s eta 0:00:01
     -------------------- ------------------- 8.3/16.3 MB 19.7 MB/s eta 0:00:01
     ---------------------------- ---------- 11.9/16.3 MB 40.9 MB/s eta 0:00:01
     --------------------------------------  16.3/16.

DEPRECATION: Loading egg at c:\users\nmadali\appdata\local\anaconda3\lib\site-packages\pyasn1_modules-0.3.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [8]:
class tokener(object):
    def __init__(self, lang):
        d = {"en":"en_core_web_sm", "fr":"fr_core_news_sm"}
        self.ob = spacy.load(d[lang])

    def tokenize(self, sent):
        sent = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sent))
        sent = re.sub(r"\!+", "!", sent)
        sent = re.sub(r"\,+", ",", sent)
        sent = re.sub(r"\?+", "?", sent)
        sent = re.sub(r"[ ]+", " ", sent)
        sent = sent.lower()
        sent = [token.text for token in self.ob.tokenizer(sent) if token.text != " "]
        sent = " ".join(sent)
        return sent

In [9]:
def dum_tokenizer(sent):
    return sent.split()

In [11]:
df = pd.read_csv(os.path.join('..',"data/", "english.txt"), names=["English"], on_bad_lines='skip')
df["French"] = pd.read_csv(os.path.join('..',"data/", "french.txt"), names=["French"], on_bad_lines='skip')["French"]


df.head()

,English,French
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Fire!,Au feu !
4,Help!,À l'aide !


In [12]:
tokenizer_fr = tokener("fr")
tokenizer_en = tokener("en")
df["English"] = df["English"].apply(lambda x: tokenizer_en.tokenize(x))
df["French"] = df["French"].apply(lambda x: tokenizer_fr.tokenize(x))


In [13]:
df.head()

,English,French
0,go .,va
1,run,cours
2,run,courez
3,fire,au feu
4,help,à l' aide


In [15]:
df.to_csv(os.path.join('..',"data", "df.csv"), index=False)

In [16]:
#!pip install torchtext==0.6

DEPRECATION: Loading egg at c:\users\nmadali\appdata\local\anaconda3\lib\site-packages\pyasn1_modules-0.3.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [17]:
import torchtext

In [19]:
FR = torchtext.data.Field(tokenize=dum_tokenizer, lower=True, init_token="<sos>", eos_token="<eos>",\
                              batch_first=True)
EN = torchtext.data.Field(tokenize=dum_tokenizer, lower=True, batch_first=True)


train = torchtext.data.TabularDataset(os.path.join('..',"data", "df.csv"), format="csv", fields=[("EN", EN), ("FR", FR)])

FR.build_vocab(train)
EN.build_vocab(train)



In [20]:
src_vocab = len(EN.vocab)
trg_vocab = len(FR.vocab)

In [21]:
src_vocab,trg_vocab

(13211, 22489)

In [22]:
sent = 'I stayed.'
sent = tokenizer_en.tokenize(sent).split()


In [23]:
sent

['i', 'stayed', '.']

In [24]:
[EN.vocab.stoi[tok] for tok in sent]

[3, 590, 2]

In [25]:
[EN.vocab.itos[tok] for tok in [3, 590, 2]]

['i', 'stayed', '.']

In [26]:
FR.vocab.stoi["<sos>"],FR.vocab.stoi["<eos>"]

(2, 3)

In [27]:
batch_size=32
train_iter = torchtext.data.BucketIterator(train, batch_size=batch_size, repeat=False, sort_key=lambda x: (len(x["EN"]), len(x["FR"])),shuffle=True, train=True)
train_length = len(train)

In [28]:
for i, data in enumerate(train_iter):
  break

In [29]:
data.FR.shape,data.EN.shape

(torch.Size([32, 21]), torch.Size([32, 12]))

In [30]:
trg_input = data.FR[:,:-1]
labels = data.FR[:,1:]

In [31]:
sample_en=data.EN[0,:].numpy()
sample_fr=data.FR[0,:].numpy()

In [32]:
sample_fr,sample_en

(array([   2,   31,   18, 3166,   62,  246,   57,  932,    4,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
       dtype=int64),
 array([  3, 211, 425, 233,  19, 386, 152,   2,   1,   1,   1,   1],
       dtype=int64))

In [33]:
[EN.vocab.itos[tok] for tok in sample_en]

['i',
 'remember',
 'having',
 'seen',
 'this',
 'movie',
 'before',
 '.',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [34]:
[FR.vocab.itos[tok] for tok in sample_fr]

['<sos>',
 'elle',
 'a',
 'accompagné',
 'son',
 'ami',
 'au',
 'concert',
 '.',
 '<eos>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']